In [1]:
import os.path as op
import os
import pandas as pd
import numpy as np
from copy import deepcopy

In [2]:
data_dirs = ['../data/results_D100', '../data/results_D25']
files = [op.join(d, f)
         for d in data_dirs
         for f in os.listdir(d)
         if f.endswith('csv')]

In [3]:
full_results = []
for fl in files:
    tmpres = pd.read_csv(fl, index_col=0)
    fname_pieces = fl.split('_')
    dset = fname_pieces[1].split('/')[0]
    experiment = fname_pieces[2]
    n_mca = int(fname_pieces[3])
    clf = fname_pieces[-4]
    dimens = fname_pieces[-2]
    preproc = fname_pieces[-3]
    random_seed = fname_pieces[-1].strip('.csv')
    resdict = {
        "Classifier": clf,
        "Experiment": experiment,
        "Simulations": n_mca,
        "Preprocessing": preproc,
        "Dimensionality": dimens,
        "Dataset": dset,
        "rs": random_seed
    }

    for idx, r in tmpres.iterrows():
        resdict_copy = deepcopy(resdict)
        # Only add ref performance for the "full" dataset
        if r.aggregation == "ref" and (n_mca != 20 or experiment != 'mca'):
            continue
        for k, v in r.items():
            resdict_copy[k] = v
        full_results += [resdict_copy]
        del resdict_copy

df_results = pd.DataFrame.from_dict(full_results)

In [4]:
n_pre = 4
n_dim = 2
n_clf = 5
n_agg_m = 6
n_agg_s = 4
n_mca = 9
n_ss = 2

n_pre*n_dim*n_clf*n_agg_m*n_mca + n_pre*n_dim*n_clf

n_pre*n_dim*n_clf*n_agg_m*n_mca + n_pre*n_dim*n_clf + n_pre*n_dim*n_clf*n_agg_s*n_ss

2520

In [5]:
df_results.query('Dataset == "D25"')

,Classifier,Experiment,Simulations,Preprocessing,Dimensionality,Dataset,rs,aggregation,acc,f1,p_acc,p_f1,test_acc,test_f1,p_test_acc,p_test_f1,test_mean_acc,test_mean_f1,n_models
2200,svc,mca,14,loggraph,fa,D25,41,median,0.700000,0.593333,0.15020,0.223600,0.600000,0.500000,0.19600,0.318,0.400000,0.320000,5
2201,rfc,mca,16,graph,fa,D25,41,mega,0.676471,0.518679,0.06960,0.228800,0.270588,0.114286,1.00000,1.000,0.402353,0.183652,5
2202,lrc,mca,18,loggraph,fa,D25,41,mean,0.650000,0.400000,0.20080,0.403200,0.400000,0.000000,0.49700,0.755,0.520000,0.380000,5
2203,ada,mca,10,zgraph,pca,D25,41,none,0.697000,0.551467,0.21756,0.266248,0.208000,0.000000,0.79952,0.734,0.325600,0.146933,250
2204,ada,mca,10,zgraph,pca,D25,41,meta,0.700000,0.560000,0.21600,0.260200,0.200000,0.000000,0.81200,0.734,0.324000,0.148000,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4715,knn,mca,2,rankgraph,pca,D25,41,none,0.650000,0.133333,0.16380,0.540600,0.400000,0.000000,0.50500,0.773,0.440000,0.000000,50
4716,knn,mca,2,rankgraph,pca,D25,41,meta,0.700000,0.660000,0.11460,0.150000,0.400000,0.000000,0.50500,0.773,0.400000,0.320000,5
4717,knn,mca,18,zgraph,pca,D25,41,mean,0.650000,0.266667,0.15780,0.454600,0.200000,0.000000,0.82000,0.755,0.520000,0.000000,5
4718,lrc,mca,20,graph,fa,D25,41,mean,0.600000,0.426667,0.23500,0.314400,0.200000,0.000000,0.79300,0.746,0.360000,0.180000,5


In [6]:
df_results.to_csv("../data/aggregation_results.csv", index=False)